# Filename: rnn_dead_reckoning_colab.ipynb

Enhanced RNN training for IMU-based dead reckoning during GPS outages.

In [ ]:
# 📦 Install dependencies (if running in Colab)
!pip install numpy pandas scikit-learn matplotlib torch --quiet


## 📁 Upload your data

In [ ]:
from google.colab import files
uploaded = files.upload()
import pandas as pd

for fn in uploaded.keys():
    print(f"Loaded: {fn}")
    df = pd.read_csv(fn)  # Assumes a .csv file


## ⚙️ Preprocessing

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler

WINDOW_SIZE = 100
STRIDE = 50

df['acc_mag'] = np.linalg.norm(df[['acc_x', 'acc_y', 'acc_z']].values, axis=1)
df['gyro_mag'] = np.linalg.norm(df[['gyro_x', 'gyro_y', 'gyro_z']].values, axis=1)
features = df[['acc_x', 'acc_y', 'acc_z', 'gyro_x', 'gyro_y', 'gyro_z', 'acc_mag', 'gyro_mag']].values
targets = df[['dx', 'dy', 'dz']].values

X, y = [], []
for i in range(0, len(df) - WINDOW_SIZE, STRIDE):
    X.append(features[i:i+WINDOW_SIZE])
    y.append(targets[i+WINDOW_SIZE-1])
X, y = np.array(X), np.array(y)

scaler = StandardScaler()
X = scaler.fit_transform(X.reshape(-1, X.shape[-1])).reshape(X.shape)

print("✅ Preprocessing complete:", X.shape, y.shape)


## 📊 Train / Validation Split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
print(f"Train size: {X_train.shape[0]}, Val size: {X_val.shape[0]}")


## 🧠 Model Definition

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout, bidirectional):
        super().__init__()
        self.rnn = nn.LSTM(input_size, hidden_size, num_layers,
                           batch_first=True, dropout=dropout, bidirectional=bidirectional)
        direction = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_size * direction, 3)

    def forward(self, x):
        _, (h_n, _) = self.rnn(x)
        if self.rnn.bidirectional:
            h_out = torch.cat((h_n[-2], h_n[-1]), dim=1)
        else:
            h_out = h_n[-1]
        return self.fc(h_out)


## 🚂 Train the Model

In [ ]:
BATCH_SIZE = 32
EPOCHS = 20
HIDDEN_SIZE = 128
DROPOUT = 0.3
NUM_LAYERS = 2
USE_BIDIRECTIONAL = True
LEARNING_RATE = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_loader = DataLoader(TensorDataset(torch.tensor(X_train).float(), torch.tensor(y_train).float()), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(TensorDataset(torch.tensor(X_val).float(), torch.tensor(y_val).float()), batch_size=BATCH_SIZE)

model = RNNModel(input_size=8, hidden_size=HIDDEN_SIZE, num_layers=NUM_LAYERS,
                 dropout=DROPOUT, bidirectional=USE_BIDIRECTIONAL).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.MSELoss()

def rmse(preds, targets):
    return torch.sqrt(criterion(preds, targets))

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        pred = model(xb)
        loss = criterion(pred, yb)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            val_loss += rmse(pred, yb).item()
    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {total_loss/len(train_loader):.4f} - Val RMSE: {val_loss/len(val_loader):.4f}")


## 📈 Plot predictions (if GPS/ground-truth is available)

In [ ]:
import matplotlib.pyplot as plt

model.eval()
X_sample = torch.tensor(X_val).float().to(device)
y_true = y_val
y_pred = model(X_sample).cpu().detach().numpy()

plt.figure(figsize=(6, 6))
plt.plot(y_true[:, 0], y_true[:, 1], label='True', alpha=0.7)
plt.plot(y_pred[:, 0], y_pred[:, 1], label='Predicted', alpha=0.7)
plt.legend()
plt.title("Displacement Prediction (X vs Y)")
plt.xlabel("dx")
plt.ylabel("dy")
plt.grid(True)
plt.axis('equal')
plt.show()
